### Making a Map

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Claudius_Ptolemy-_The_World.jpg/600px-Claudius_Ptolemy-_The_World.jpg)

**OBJECTIVES**:

- Use `folium` to make basic maps
- Add markers to maps with popups
- Make choropleth maps to display state wide data


### Introduction to `folium`

There are a few options for making maps with Python -- folium has been the easiest to get up and running with so we will start here. 

```
pip install folium
```

In [1]:
# pip install -U folium

In [2]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import numpy as np

In [3]:
# make a map
folium.Map()

In [4]:
# find lat/long of your hometown -- use as location for new map
hometown = [42.8142, -73.9396]
m = folium.Map(location = hometown)

In [5]:
m

In [6]:
# zoom start at 5
m2 = folium.Map(location=hometown, zoom_start = 15)
m2

In [7]:
# different style with tiles 
m3 = folium.Map(location = hometown, tiles = "Stamen Toner")
m3

### Using Markers

We can easily add markers to the map and style these.  To do so we create the marker, and add it to the map object. 

In [8]:
# create a marker at your hometown latitude and logitude
Marker(location = hometown).add_to(m3)

In [9]:
m3

In [10]:
# add popup with name information to the marker
Marker(location = hometown, popup = "<strong>Who</strong>: Jacob.").add_to(m2)
m2

In [11]:
# try a circle marker
Circle(location = hometown, radius = 10).add_to(m2)
m2

**NOTE**

Sometimes we want to loop over a DataFrame.  This is not usually a good idea, but sometimes it is the clearest solution to our problem.  To do so, we can use `.iterrows` which returns the row index and columns for the given row.

In [12]:
ex_df = pd.DataFrame({'state': ['NY', 'MA', 'CT'],
                     'lat': [20, 50, 30],
                     'long': [87, 91, 77],
                     'names': ['Lenny', 'Hardy', 'Tino']})

In [13]:
#loop over iterrows
for idx, row in ex_df.iterrows():
    print(idx)
    lat = row['lat']
    long = row['long']
    print(lat, long)

0
20 87
1
50 91
2
30 77


In [14]:
#loop over rows and print names
for _, row in ex_df.iterrows():
    print(row['names'])

Lenny
Hardy
Tino


In [15]:
_

2

#### Marker Clusters

With a large number of points, you may want to use the `MarkerCluster` method.  

In [16]:
#read in chicago crime data
chicago = pd.read_csv('data/chicago_crimes.csv')
chicago_nona = chicago.dropna(subset = ['latitude', 'longitude'])
chicago_nona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9913 entries, 1 to 9999
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         9913 non-null   object 
 1   block                        9913 non-null   object 
 2   description                  9913 non-null   object 
 3   location_description         9913 non-null   object 
 4   iucr                         9913 non-null   object 
 5   ward                         9913 non-null   int64  
 6   year                         9913 non-null   int64  
 7   case_number                  9913 non-null   object 
 8   fbi_code                     9913 non-null   object 
 9   domestic                     9913 non-null   bool   
 10  beat                         9913 non-null   int64  
 11  arrest                       9913 non-null   bool   
 12  primary_type                 9913 non-null   object 
 13  :@computed_region_

In [17]:
#create a map
chicago_nona[['latitude', 'longitude']].head(1)

,latitude,longitude
1,41.90004,-87.723783


In [18]:
cm = folium.Map(location = [41.90004,-87.723783] )
cm

In [19]:
#create a marker cluster object
marker_clust = MarkerCluster(locations=chicago_nona[['latitude', 'longitude']])

In [20]:
#iterate over points and add_child to cluster
cm.add_child(marker_clust)

In [21]:
chicago_nona.head(2)

,date,block,description,location_description,iucr,ward,year,case_number,fbi_code,domestic,...,y_coordinate,:@computed_region_rpca_8um6,latitude,:@computed_region_awaf_s7ux,x_coordinate,longitude,:@computed_region_bdys_3d7i,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_d3ds_rm58
1,2012-01-01T00:00:00.000,010XX N SPRINGFIELD AVE,PREDATORY,RESIDENCE,0266,27,2012,HY448381,02,True,...,1906754.0,5.0,41.90004,41.0,1150193.0,-87.723783,456.0,4299.0,24.0,66.0
3,2012-01-01T00:00:00.000,067XX S RIDGELAND AVE,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,1754,5,2012,HY489463,02,True,...,1860569.0,24.0,41.77246,32.0,1189029.0,-87.582622,381.0,22538.0,39.0,212.0


In [22]:
#add_child cluster to the map
cm2 = folium.Map(location = [41.90004,-87.723783] )
for _, row in chicago_nona.head(20).iterrows():
    lat = row['latitude']
    long = row['longitude']
    desc = row['description']
    Marker(location = [lat, long], popup = desc).add_to(cm2)

In [23]:
cm2

**Problem**

Mapping arrest data.  Below, you are given the url to data from New York City about arrests.  Your goal is to make a map with markers for the arrest data with information about the offense description in the marker popup.

In [24]:
url = 'https://data.cityofnewyork.us/resource/uip8-fykc.json'

In [25]:
nyc_crime = pd.read_json(url)
nyc_crime.head(2)

,arrest_key,arrest_date,pd_desc,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,...,latitude,longitude,geocoded_column,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_efsh_h5xi,pd_cd,ky_cd
0,239406147,2022-01-20T00:00:00.000,(null),(null),CPL5700600,9,Q,113,3,25-44,...,40.679701,-73.776047,"{'type': 'Point', 'coordinates': [-73.77604735...",41,3,46,71,24669.0,NaN,NaN
1,239828064,2022-01-28T00:00:00.000,(null),(null),CPL5700600,9,Q,113,3,45-64,...,40.679701,-73.776047,"{'type': 'Point', 'coordinates': [-73.77604735...",41,3,46,71,24669.0,NaN,NaN


In [26]:
nyc_map = folium.Map(location = [40.679701, -73.776047])
for _, row in nyc_crime.iterrows():
    lat = row['latitude']
    long = row['longitude']
    desc = row['ofns_desc']
    Marker(location = [lat, long], popup = desc).add_to(nyc_map)

In [27]:
nyc_map

### Using Boundaries

Now, we will load in a dataset containing boundaries for states.  This will allow us to create a `choropleth` map that colors inside the state boundaries. 

In [28]:
#link to folium data
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

In [29]:
#check out the data
state_geo

'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'

In [30]:
# read in the state data
state_data = pd.read_csv('data/state_unemployment.csv')

In [31]:
# take a look at columns
state_data.head()

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1


In [32]:
#map 
m = folium.Map(location=[48, -102], zoom_start=3)
#choropleth layer
folium.Choropleth(
    geo_data=state_geo,     #geo json data
    name="choropleth",
    data=state_data,        #dataframe with info
    columns=["State", "Unemployment"], #column to link to geo json and to color by
    key_on="feature.id", #connect to state in geo json
    legend_name="Unemployment Rate (%)",
).add_to(m)



m

### COVID Map

Below, you are given statewide data on COVID cases.  Use the `Province_State` column to create a choropleth map with whatever data you would like -- Deaths, Recovered, People Hospitalized.  Do you see any patterns?  Be prepared to share your code in slack.

In [33]:
cdf = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/10-10-2022.csv')

In [34]:
cdf.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2022-10-11 04:31:31,32.3182,-86.9023,1525724,20473,NaN,NaN,1.0,31116.998441,NaN,NaN,1.341855,84000001.0,USA,NaN,NaN
1,Alaska,US,2022-10-11 04:31:31,61.3707,-152.4044,298869,1393,NaN,NaN,2.0,40854.492888,NaN,NaN,0.466090,84000002.0,USA,NaN,NaN
2,American Samoa,US,2022-10-11 04:31:31,-14.2710,-170.1320,8250,34,NaN,NaN,60.0,14827.195773,NaN,NaN,0.412121,16.0,ASM,NaN,NaN
3,Arizona,US,2022-10-11 04:31:31,33.7298,-111.4312,2275235,31406,NaN,NaN,4.0,31258.736945,NaN,NaN,1.380341,84000004.0,USA,NaN,NaN
4,Arkansas,US,2022-10-11 04:31:31,34.9697,-92.3731,954177,12321,NaN,NaN,5.0,31618.256189,NaN,NaN,1.291270,84000005.0,USA,NaN,NaN


In [35]:
#columns = ['Province_State, 'Deaths']
#key_on = 'feature.properties.name'




#### Exit Exercise

Check out the SEC API and its Python wrapper [here](https://sec-api.io/docs/executive-compensation-api/python-example) to see how to get the executive compensation for a given company or companies.  Create a visualization of your choice using the data you retreive. 

In [36]:
folium.__version__

'0.13.0'